# PySCF Library Overview

This code utilizes the PySCF (Python-based Simulations of Chemistry Framework), an open-source computational chemistry library written in Python, designed for electronic structure calculations, particularly in quantum chemistry. PySCF supports a wide range of methods, including Hartree-Fock (HF), Density Functional Theory (DFT), Møller–Plesset perturbation theory (MP2), Coupled Cluster (CC), and Configuration Interaction (CI). It allows users to easily combine different computational methods and customize their workflows. PySCF is optimized for performance, utilizing efficient algorithms and parallel computing to handle large-scale calculations. It is also designed to be extensible, enabling researchers to implement and test new methods and algorithms with minimal effort. Common applications of PySCF include electronic structure calculations, molecular property computations (such as dipole moments, polarizabilities, and vibrational frequencies), studying reaction mechanisms, and investigating the electronic properties of solids and nanostructures.


In [ ]:
import pyscf
from pyscf import gto, scf, tdscf, cc, tddft, ao2mo
from pyscf.tools import molden
import matplotlib.pyplot as plt
import numpy as np
import scipy

In [ ]:
# Create the molecule
mol = gto.Mole()
# water
mol.atom = '''
O          0.00000        0.00000        0.11779
H          0.00000        0.75545       -0.47116
H          0.00000       -0.75545       -0.47116'''
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()
mf = scf.RHF(mol).run(verbose = 4)
mf.analyze()

# Quantum Chemistry and the Hamiltonian

Solving the Hamiltonian is challenging even for the single-electron case. For systems with three or more electrons, considering all possible interactions becomes essential. A molecule comprises numerous atoms, each containing multiple subatomic particles, including electrons. Therefore, in addition to the multitude of variables and the complexity of the equations, the sheer dimension of the Hamiltonian required to account for all possible configurations is already too large to solve with simple calculations. Quantum chemistry has made significant progress in addressing such challenges. *Ab initio* methods, also known as wave function methods, aim to solve the Schrödinger equation from the beginning, constructing approximate wave functions based on the positions of nuclei and the number of electrons.

## Time-Independent Schrödinger Equation (TISE)

For simplicity, let's consider the time-independent Schrödinger equation (TISE), as shown in Equation (1), with a simplified electronic Hamiltonian depicted below:

$$
\hat{H}\ket{\Psi} = E \ket{\Psi} \tag{1}
$$

Assuming that the potential associated with the system is time-independent, we can apply the Born-Oppenheimer approximation, which assumes that the electrons move much faster than the nuclei, allowing us to treat the nuclear coordinates as fixed. In this simplification, hyperfine interactions are ignored, and the electronic states depend on electrons and parametrically on nuclear coordinates.

## Born-Oppenheimer Approximation and Electronic Hamiltonian

The electronic Hamiltonian under the Born-Oppenheimer approximation is given by:

$$
\hat{H}(r,R) = - \sum_{i=1}^N \frac{1}{2}\nabla_i^2 - \sum_{i=1}^N \sum_{A=1}^M \frac{Z_A}{r_{iA}} + \sum_{i=1}^N \sum_{j>i}^N \frac{1}{r_{ij}} \tag{2}
$$

Where:
- $ \hat{H}(r,R) $ is the electronic Hamiltonian,
- $ r $ and $ R $ represent the electronic and nuclear coordinates respectively,
- $ N $ is the number of electrons,
- $ M $ is the number of nuclei,
- $ \nabla_i^2 $ is the Laplacian operator acting on the $ i $-th electron,
- $ Z_A $ is the atomic number of nucleus $ A $,
- $ r_{iA} $ is the distance between electron $ i $ and nucleus $ A $,
- $ r_{ij} $ is the distance between electrons $ i $ and $ j $.

In this Hamiltonian:
- The first term $ - \sum_{i=1}^N \frac{1}{2}\nabla_i^2 $ represents the kinetic energy of the electrons.

- The second term $ - \sum_{i=1}^N \sum_{A=1}^M \frac{Z_A}{r_{iA}} $ represents the Coulomb attraction between the electrons and nuclei.

- The third term $ \sum_{i=1}^N \sum_{j>i}^N \frac{1}{r_{ij}} $ represents the Coulomb repulsion between electrons.

This formulation allows us to approximate the electronic wave function while treating the nuclear positions as fixed, greatly simplifying the computational complexity involved in solving the Schrödinger equation for multi-electron systems.


## Slater determinant

In the simplified electronic Hamiltonian shown in equations \ref{electronicWFNfirst}, a single electron's behavior is described by its spatial wavefunction, $\phi(r)$, where $r$ includes Cartesian coordinates $x$, $y$, $z$, and its spin state, denoted by $w$ (either $\alpha$ or $\beta$). A spin-orbital, denoted as $\chi(r,w)$, combines spatial and spin coordinates as a product of functions. For a system of $N$ non-interacting electrons distributed across $K$ atomic orbitals, the Hamiltonian can be represented as the sum of single-electron Hamiltonians for individual electrons. The eigenfunction of this system is a product of spin orbitals for each electron, known as the Hartree product and denoted as $\Psi^{HP}$

$$
    \Psi^{HP}(x_1, x_2, \cdots, x_N) = \chi_i(x_1)\chi_j(x_2)\cdots\chi_k(x_N)
$$

Electrons follow the Pauli exclusion principle, where the electronic wave functions must exhibit antisymmetry upon the exchange of any two particles. This antisymmetric property can be achieved with the Slater determinant, $\ket{\Psi^{SD}}$, as shown in eq~\eqref{sdwfn}. The Slater determinant is defined as a linear combination of Hartree products, mathematically expressed in terms of matrix determinants. It is the simplest antisymmetric wave function that can describe the ground state of an $N$-electron system.

$$
    \ket{\Psi^{SD}} = \frac{1}{\sqrt{N!}} \begin{vmatrix}  \chi_i(x_1) & \dots &\chi_k(x_1) \\ \vdots & \ddots & \vdots \\ \chi_i(x_N) &  \dots & \chi_k(x_N)  \end{vmatrix}= \ket{\chi_i\chi_j\cdots}
$$


## Hartree-Fock Method

The Hartree-Fock (HF) approach is the starting point for many \emph{ab initio} methods. It provides an approximation to the many-body problems posed by an N-electron system through single-particle states such that the solution can be a single Slater determinant. The single-particle states, $\{\chi_i\}$, can be chosen to be orthonormal. Then, using Lagrange's method of undetermined multipliers, with the constraint of orthonormality between spin orbitals and differentiating with respect to the molecular coefficients, it minimizes the expectation value of the Hamiltonian, providing the optimal set of orbitals with the lowest energy, $\epsilon_{ij}$ following eq~\eqref{canHF}. 

$$
    \sum_i\epsilon_{ij} \chi_i(1) = \left[\hat{h} + \sum_j \int dx_2\chi_j(2)\frac{1}{r_{12}}\chi_j(2) - \sum_j \int dx_2\chi_j(2)\frac{1}{r_{12}}\hat{\mathcal{P}}\chi_j(2)\right] \chi_i(1)
$$

Here, $\hat{\mathcal{P}}$ is an operator that interchanges electrons, for example, $\hat{\mathcal{P}} \chi_j(2) \chi_i(1) = \chi_i(2)\chi_j(1)$. Hartree-Fock (HF) is a mean-field method that captures the electron-electron interaction between the $i$-th and $j$-th electrons as an average potential from the other electrons. The second term in the second part of eq~\eqref{canHF} accounts for the Coulombic interaction, while the third term accounts for the exchange interaction arising from the antisymmetric nature of fermions. Therefore, we can define the Fock operator ($\hat{f}$), which combines the kinetic and potential energies of electrons as described by the singlet electron Hamiltonian ($\hat{h}$), along with operators for the Coulombic interactions between electrons ($\hat{J}$) and the exchange interactions ($\hat{K}$). For electron k, eq~\eqref{canHF} in terms of operators is written as follows:

$$
   \hat{f}(k) = \hat{h}(k) + \sum_{j=1}^{N} (\hat{J}_j(k) - \hat{K}_j(k)) 
$$

$$
    \hat{J}_j(1) = \sum_{j} \int dx_2\chi^*_j(2)\frac{1}{r_{12}}\chi_j(2) 
$$

$$
  \hat{K}_j(1) = \sum_{j} \int dx_2\chi^*_j(2)\frac{1}{r_{12}}\hat{\mathcal{P}}\chi_j(2)
$$


In [ ]:
# Perform Restricted Hartree-Fock calculation with PySCF
mf = scf.RHF(mol).run(verbose = 4)
mf.analyze()

H_core = mf.get_hcore()
e_hf = mf.kernel()

F = mf.get_fock()
J, K = mf.get_jk()
D = mf.make_rdm1()
S = mf.get_ovlp()

Vsys = mf.get_veff()
mo_energy = mf.mo_energy
mo_occ = mf.get_occ(mo_energy)
C = mf.mo_coeff

n_elec = 2*round(0.5 * np.trace(D@S))
num_orbitals = len(C)

## Configuration Interaction (CI) Methods

The Configuration Interaction (CI) methods involve a linear combination of different Slater determinants. The Full CI wave function, as depicted in eq~\eqref{fciwfn}, encompasses linear combinations of all possible configurations and is therefore considered an exact solution within a given basis set. Here, $\Psi_0$ represents the ground state wave function, while $\Psi_i^a$ denotes determinants within the Fock space with singly excited states, achieved by switching the $i^{th}$ orbital with the $a^{th}$ orbital.

$$
  \ket{\Psi^{FCI}} = c_{0}\ket{\Psi^{(0)}} +\sum_{i,a} c_{i}^{a}\ket{\Psi_i^a} +\sum_{ijab}c_{ij}^{ab}\ket{\Psi_{ij}^{ab}}+...   
$$

Similar to the HF, the total CI wave functions are set to be orthonormal to each other; using a Lagrange multiplier with this constraint, the CI energy is to be obtained variationally. By optimizing expansion coefficients, one can lower the energy below the HF energy. Matrix elements can be evaluated with Slater's rule, and the matrix representation of the CI equation becomes eq~\eqref{ci} where the solution is equivalent to the diagonalization of the CI matrix.

$$
Hc = ESc
$$


## Tamm-Dancoff Approximation (TDA) in Density Functional Theory (DFT)

The Tamm-Dancoff Approximation (TDA) is a simplified approach in the Time-Dependent Density Functional Theory (TDDFT). Despite some differences in implementation, both methods become analogous when it comes to Configuration Interaction Singles (CIS) and share the common goal of improving the accuracy of excited state calculations by considering electron interactions beyond the mean-field approximation.

### References

Dreuw, A., & Head-Gordon, M. (2005). Single-Reference Ab Initio Methods for the Calculation of Excited States of Large Molecules. *Chem. Rev.*, 105(11), 4009-4037. doi: [10.1021/cr0505627](https://pubs.acs.org/doi/10.1021


In [ ]:
def tda_denisty_matrix(td, state_id):
    # ‘’'
    # Taking the TDA amplitudes as the CIS coefficients, calculate the density
    # matrix (in AO basis) of the excited states
    # ‘’'
    cis_t1 = td.xy[state_id][0]
    dm_oo =-np.einsum('ia,ka->ik', cis_t1.conj(), cis_t1)
    dm_vv = np.einsum('ia,ic->ac', cis_t1, cis_t1.conj())
    # The ground state density matrix in mo_basis
    mf = td._scf
    dm = np.diag(mf.mo_occ)
    # Add CIS contribution
    nocc = cis_t1.shape[0]
    # Note that dm_oo and dm_vv correspond to spin-up contribution. “*2” to
    # include the spin-down contribution
    dm[:nocc,:nocc] += dm_oo * 2
    dm[nocc:,nocc:] += dm_vv * 2
    # Transform density matrix to AO basis
    mo = mf.mo_coeff
    dm = np.einsum('pi,ij,qj->pq', mo, dm, mo.conj())
    return dm

In [ ]:
#TDA
tda_h2o = tdscf.TDA(mf)
tda_h2o.nstates = 1
e_s_cis = min(tda_h2o.kernel()[0])

tda_h2o = tdscf.TDA(mf)
tda_h2o.nstates = 1
tda_h2o.singlet = False
e_t_cis = min(tda_h2o.kernel()[0])

### Multi-configuration methods

Active space methods not only provide a well-established solution for static correlation problems but also enhance computational efficiency. By partitioning the orbitals into three distinct spaces, calculations can exclusively focus on the relevant orbitals. The classification of these spaces determines the specific approach utilized.


In [ ]:
act_list = []
doc_list = []

# Calculate HOMO and LUMO indices from mo_occ
homo_index = np.where(mo_occ == 2)[0][-1]
lumo_index = homo_index + 1

print('homoidx',homo_index)
print('lumoidx',lumo_index)

#active_sizes = len(lumo_index,num_orbitals)
#active_sizes = list(range(0,active_sizes))

active_sizes = []
elists = []
elistt = []
hf = []

w = round(num_orbitals - 0.5 * n_elec)

In [ ]:
for i in range(0, w -1):
    active_sizes.append(2*i)

    occ_list = list(range(0,homo_index-i))
    act_list = list(range(homo_index - i, lumo_index+i+1))
    vir_list = list(range(lumo_index + i +1, num_orbitals))

    print('actlist',act_list)
    act_array = np.array(act_list)

    Cocc = C[:, occ_list]
    occ_occ = mo_occ[occ_list]
    occ_occ = np.array(occ_occ)

    Cact = C[:, act_list]
    act_occ = mo_occ[act_list]
    act_occ = np.array(act_occ)

   #molden.from_mo(mol, 'ttc_act.molden',Cact)

    Cvir = C[:, vir_list]
    vir_occ = mo_occ[vir_list]
    vir_occ = np.array(vir_occ)

    nact = len(act_list)
    nvir = len(vir_list)
    nocc = len(occ_list)

    D_O = np.dot(Cocc*occ_occ, Cocc.conj().T)
    D_A = np.dot(Cact*act_occ, Cact.conj().T)
    D_C = np.dot(Cvir*vir_occ, Cvir.conj().T)

    D_tot = D_A +D_C +D_O

    P_c = np.dot(Cvir, Cvir.conj().T)
    P_o = np.dot(Cocc, Cocc.conj().T)

    #projector
    P = S @ P_c @ S + S@P_o@S
    mu = 1.0e6

    Vsys = mf.get_veff(dm=D_tot)
    Vact = mf.get_veff(dm=D_A) 
    Venv = mf.get_veff(dm=D_C)

    #new fock ao
    Vemb = Vsys - Vact + (mu * P)
    verror = Vsys - Vact


    n_act = 2*round(0.5 * np.trace(S@D_A))

    print('Number of Occ orbitals: ', nocc)
    print('Number of Active orbitals: ', nact)
    print('Number of Virtual orbitals: ', nvir) 
    print('Number of electrons in active space',n_act) 

    emb_mf = scf.RHF(mol)
    mol.nelectron = n_act
    mol.build()

    emb_mf.verbose = 4
    emb_mf.get_hcore = lambda *args: H_core + Vemb
    emb_mf.max_cycle = 200
    e_hf_act = emb_mf.kernel(dm0=D_A)
    print('ehfact',e_hf_act)

    emb_tda = tdscf.TDA(emb_mf)
    emb_tda.nstates = 3
    e = min(emb_tda.kernel()[0])
    e_ev = e * ev_conversion_factor

    emb_tda_t = tdscf.TDA(emb_mf)
    emb_tda_t.nstates = 3
    emb_tda_t.singlet = False
    e_t = min(emb_tda_t.kernel()[0])
    e_t_ev = e_t * ev_conversion_factor

    elists.append(e_ev)
    elistt.append(e_t_ev)


In [ ]:
active_sizes_can = np.asarray(active_sizes)
elists_no = np.asarray(elists)
np.savetxt("can_HL_singlet", elists, fmt='%1.13f')

elistt_no = np.asarray(elistt)
np.savetxt("can_HL_triplet", elistt, fmt='%1.13f')

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.plot(active_sizes_can, elists_no, marker='o', linestyle='-', label='CIS_act')
plt.axhline(y=e_s_cis, color='blue', linestyle='--', label='CIS')
# plt.axhline(y=e_s, color='black', linestyle='--', label='EOM-CCSD')
# plt.axhline(y=e_dft_s, color='red', linestyle='--', label='TDDFT')
plt.xlabel('# of orbitals in active space')
plt.ylabel('Excited State energies (eV)')
plt.title('Excitation energy of active space; Singlets')
plt.grid(True)
plt.legend()
plt.tight_layout()

plt.subplot(1, 2, 2)
plt.plot(active_sizes_can, elistt_no, marker='o', linestyle='-', label='CIS_act')
plt.axhline(y=e_t_cis, color='blue', linestyle='--', label='CIS')
# plt.axhline(y=e_t, color='black', linestyle='--', label='EOM-CCSD')
# plt.axhline(y=e_dft_t, color='red', linestyle='--', label='TDDFT')
plt.xlabel('# of orbitals in active space')
plt.ylabel('Excited State energies (eV)')
plt.title('Excitation energy of active space; Triplets')
plt.grid(True)
plt.legend()
plt.tight_layout()

file_name = "no_HL_plots.png"
plt.savefig(file_name)
plt.show()


## References
1. Szabo, A.; Ostlund, N. S. *Modern Quantum Chemistry*; McGraw-Hill: New York, 1989.
2. Sun, Q., Berkelbach, T. C., Blunt, N. S., Booth, G. H., Guo, S., Li, Z., Liu, J., McClain, J. D., Sayfutyarova, E. R., Sharma, S., Wouters, S., Chan, G. K.-L. (2017). PySCF: the Python-based simulations of chemistry framework. *WIREs Computational Molecular Science*, 8(1), e1340. doi:10.1002/wcms.1340
